In [1]:
import torch
print(torch.version.cuda)
import torchvision
print(torchvision.__version__)
torch.cuda.empty_cache()

C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


11.6
0.13.1+cpu


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
from carbontracker.tracker import CarbonTracker
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Feb 21 10:32:51 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 472.12       Driver Version: 472.12       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
|  0%   33C    P8    14W / 310W |    798MiB /  8192MiB |     18%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount = True)

In [4]:
# !7z x <> -o '/content/drive/MyDrive/Colab Notebooks/465/dynamic_rectification_knowledge_distillation/data'

In [5]:
# setting device on GPU if available, else CPU
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cuda

NVIDIA GeForce RTX 3070 Ti
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [6]:
# %cd '/content/drive/MyDrive/Colab Notebooks/465/dynamic_rectification_knowledge_distillation'

In [7]:
# !pip install absl-py
# !pip install astor
# !pip install certifi
# !pip install cffi
# !pip install chardet
# !pip install ConfigArgParse
# !pip install future
# !pip install gast
# !pip install grpcio
# !pip install idna
# !pip install Markdown
# !pip install mkl-fft
# !pip install mkl-random
# !pip install numpy
# !pip install olefile
# !pip install Pillow
# !pip install protobuf
# !pip install pycparser
# !pip install requests
# !pip install scipy
# !pip install six
# !pip install tb-nightlya
# !pip install tensorboard
# !pip install tensorboardX
# !pip install tensorflow
# !pip install termcolor
# !pip install torch
# !pip install torchtext
# !pip install torchvision
# !pip install tqdm
# !pip install urllib3
# !pip install Werkzeug

# !pip install -U scikit-image
# !pip install ipykernel

In [8]:
!pip install opencv-python


[notice] A new release of pip available: 22.2.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
from model import abc #Tiny Imagenet Custom Dataset seems to cause unseen multi-processing error

In [10]:
from torch.utils.data import Dataset, DataLoader
from skimage import io
import cv2
import glob
import random

import torchvision.transforms as transforms



data_transforms = {
            'train': transforms.Compose([
                transforms.ToPILImage(),
                transforms.RandomRotation(20),
                transforms.RandomHorizontalFlip(0.5),
                transforms.ToTensor(),
                transforms.Normalize([0.4802, 0.4481, 0.3975], [
                                     0.2302, 0.2265, 0.2262]),
            ]),
            'val': transforms.Compose([
                transforms.ToPILImage(),
                transforms.ToTensor(),
                transforms.Normalize([0.4802, 0.4481, 0.3975], [
                                     0.2302, 0.2265, 0.2262]),
            ])
        }


####################################################
#       Create Train and Val sets
####################################################
def flatten(t):
    return [item for sublist in t for item in sublist]


def generate_train_val_image_path():
    train_data_path = '.\\data\\tiny-imagenet-200\\train'
    val_data_path = '.\\data\\tiny-imagenet-200\\val'

    train_image_paths = []  # to store image paths in list
    classes = []  # to store class values

    for data_path in glob.glob(train_data_path + '\\*'):
        classes.append(data_path.split('\\')[-1])
        train_image_paths.append(glob.glob(data_path + '\\*'))
        
    train_image_paths = list(flatten(train_image_paths))
    random.shuffle(train_image_paths)

    print('train_image_path example: ', train_image_paths[0])
#     print('class example: ', classes[0])

    # 3.
    # create the val
    val_image_paths = []
    for data_path in glob.glob(val_data_path + '\\images'):
        val_image_paths.append(glob.glob(data_path + '\\*\\*'))

    val_image_paths = list(flatten(val_image_paths))

    print("Train size: {}\nValid size: {}\n".format(len(train_image_paths), len(val_image_paths)))
#     print('val_image_path example: ', val_image_paths[0])
#     print('class example: ', classes[0])
    idx_to_class = {i: j for i, j in enumerate(classes)}
    class_to_idx = {value: key for key, value in idx_to_class.items()}

    return train_image_paths, val_image_paths, idx_to_class, class_to_idx, classes




#######################################################
#               Define Dataset Class
#######################################################

# class TinyImagenetDataset(Dataset):
#     def __init__(self, image_paths, class_to_idx, use_cache = True, cache_size = 50000 ,transform = None):
#         self.image_paths = image_paths
#         self.transform = transform
#         self.cached_data = []
#         self.cache_size = cache_size
#         self.cache = {}
#         self.class_to_idx = class_to_idx

#     def __len__(self):
#         return len(self.image_paths)

#     def __getitem__(self, index):
#         if index in self.cache:
#             image, label = self.cache[index]
#         else:
#             image_filepath = self.image_paths[index]
#             image_filepath = image_filepath.replace('\\', '/')
#             image = io.imread(image_filepath) # your slow data loading
#             image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#             label = image_filepath.split('/')[-2]
#             label = self.class_to_idx[label]
#             if len(self.cache) < self.cache_size:
#                 self.cache[index] = (image, label)
        
        
#         if self.transform is not None:
#             image = self.transform(image)

#         return image, label

def get_final_train_and_test_set():
    train_image_paths, val_image_paths, idx_to_class, class_to_idx, classes = generate_train_val_image_path()

    trainset = abc.TinyImagenetDataset(train_image_paths, class_to_idx, transform=data_transforms['train'])
#     print(train_image_paths)
    devset = abc.TinyImagenetDataset(val_image_paths, class_to_idx, transform=data_transforms['val'])
#     print(val_image_paths)
    return trainset, devset

In [11]:
##################################################################
#   Utils
##################################################################


"""
Tensorboard logger code referenced from:
https://github.com/yunjey/pytorch-tutorial/blob/master/tutorials/04-utils/
Other helper functions:
https://github.com/cs230-stanford/cs230-stanford.github.io
"""

import json
import logging
import os
import shutil
import torch
from collections import OrderedDict
from torch.optim.lr_scheduler import _LRScheduler
import tensorflow as tf
import numpy as np
import scipy.misc 
try:
    from StringIO import StringIO  # Python 2.7
except ImportError:
    from io import BytesIO         # Python 3.x


class Params():
    """Class that loads hyperparameters from a json file.

    Example:
    ```
    params = Params(json_path)
    print(params.learning_rate)
    params.learning_rate = 0.5  # change the value of learning_rate in params
    ```
    """

    def __init__(self, json_path):
        with open(json_path) as f:
            params = json.load(f)
            self.__dict__.update(params)

    def save(self, json_path):
        with open(json_path, 'w') as f:
            json.dump(self.__dict__, f, indent=4)
            
    def update(self, json_path):
        """Loads parameters from json file"""
        with open(json_path) as f:
            params = json.load(f)
            self.__dict__.update(params)

    @property
    def dict(self):
        """Gives dict-like access to Params instance by `params.dict['learning_rate']"""
        return self.__dict__


class RunningAverage():
    """A simple class that maintains the running average of a quantity
    
    Example:
    ```
    loss_avg = RunningAverage()
    loss_avg.update(2)
    loss_avg.update(4)
    loss_avg() = 3
    ```
    """
    def __init__(self):
        self.steps = 0
        self.total = 0
    
    def update(self, val):
        self.total += val
        self.steps += 1
    
    def __call__(self):
        return self.total/float(self.steps)

class AverageMeter(object):

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val*n
        self.count += n
        self.avg = self.sum/self.count

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def set_logger(log_path):
    """Set the logger to log info in terminal and file `log_path`.

    In general, it is useful to have a logger so that every output to the terminal is saved
    in a permanent file. Here we save it to `model_dir/train.log`.

    Example:
    ```
    logging.info("Starting training...")
    ```

    Args:
        log_path: (string) where to log
    """
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)

    if not logger.handlers:
        # Logging to a file
        file_handler = logging.FileHandler(log_path)
        file_handler.setFormatter(logging.Formatter('%(asctime)s:%(levelname)s: %(message)s'))
        logger.addHandler(file_handler)

        # Logging to console
        stream_handler = logging.StreamHandler()
        stream_handler.setFormatter(logging.Formatter('%(message)s'))
        logger.addHandler(stream_handler)


def save_dict_to_json(d, json_path):
    """Saves dict of floats in json file

    Args:
        d: (dict) of float-castable values (np.float, int, float, etc.)
        json_path: (string) path to json file
    """
    with open(json_path, 'w') as f:
        # We need to convert the values to float for json (it doesn't accept np.array, np.float, )
        d = {k: float(v) for k, v in d.items()}
        json.dump(d, f, indent=4)


def save_checkpoint(state, is_best, checkpoint, epoch_checkpoint = False):
    """Saves model and training parameters at checkpoint + 'last.pth.tar'. If is_best==True, also saves
    checkpoint + 'best.pth.tar'

    Args:
        state: (dict) contains model's state_dict, may contain other keys such as epoch, optimizer state_dict
        is_best: (bool) True if it is the best model seen till now
        checkpoint: (string) folder where parameters are to be saved
    """
    filepath = os.path.join(checkpoint, 'last.pth.tar')
    if not os.path.exists(checkpoint):
        print("Checkpoint Directory does not exist! Making directory {}".format(checkpoint))
        os.mkdir(checkpoint)
    else:
        print("Checkpoint Directory exists! ")
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'best.pth.tar'))
    if epoch_checkpoint == True:
        epoch_file = str(state['epoch']-1) + '.pth.tar'
        shutil.copyfile(filepath, os.path.join(checkpoint, epoch_file))





def load_checkpoint(checkpoint, model, optimizer=None):
    """Loads model parameters (state_dict) from file_path. If optimizer is provided, loads state_dict of
    optimizer assuming it is present in checkpoint.

    Args:
        checkpoint: (string) filename which needs to be loaded
        model: (torch.nn.Module) model for which the parameters are loaded
        optimizer: (torch.optim) optional: resume optimizer from checkpoint
    """
    try:

      if not os.path.exists(checkpoint):
        raise FileNotFoundError
    except FileNotFoundError:
      ("File doesn't exist {}".format(checkpoint))

    if torch.cuda.is_available():
        checkpoint = torch.load(checkpoint)
    else:
        # this helps avoid errors when loading single-GPU-trained weights onto CPU-model
        checkpoint = torch.load(checkpoint, map_location=lambda storage, loc: storage)

    model.load_state_dict(checkpoint['state_dict'])

    if optimizer:
        optimizer.load_state_dict(checkpoint['optim_dict'])

    return checkpoint


class WarmUpLR(_LRScheduler):
    """warmup_training learning rate scheduler
    Args:
        optimizer: optimzier(e.g. SGD)
        total_iters: totoal_iters of warmup phase
    """

    def __init__(self, optimizer, total_iters, last_epoch=-1):
        self.total_iters = total_iters
        super().__init__(optimizer, last_epoch)

    def get_lr(self):
        """we will use the first m batches, and set the learning
        rate to base_lr * m / total_iters
        """
        return [base_lr * self.last_epoch / (self.total_iters + 1e-8) for base_lr in self.base_lrs]

In [12]:
##################################################################
#   Data Loader
##################################################################

"""
   CIFAR-10 CIFAR-100, Tiny-ImageNet data loader
"""

import os
import random

import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data.sampler import SubsetRandomSampler

def fetch_dataloader(types, params, dataset_name=None):
    """
    Fetch and return train/dev dataloader with hyperparameters (params.subset_percent = 1.)
    """
    # using random crops and horizontal flip for train set
    if params.augmentation == "yes":
        train_transformer = transforms.Compose([
            transforms.RandomCrop(
                32, padding=4),
            transforms.RandomHorizontalFlip(),  # randomly flip image horizontally
            transforms.RandomRotation(15),
            transforms.ToTensor(),
            transforms.Normalize((0.5070751592371323, 0.48654887331495095, 0.4409178433670343),
                                 (0.2673342858792401, 0.2564384629170883, 0.27615047132568404))])
        # transforms.Normalize((0.4914, 0.4822, 0.4465), (0.240, 0.243, 0.261))

    # data augmentation can be turned off
    else:
        train_transformer = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5070751592371323, 0.48654887331495095, 0.4409178433670343),
                                 (0.2673342858792401, 0.2564384629170883, 0.27615047132568404))])

    # transformer for dev set
    dev_transformer = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5070751592371323, 0.48654887331495095, 0.4409178433670343),
                             (0.2673342858792401, 0.2564384629170883, 0.27615047132568404))])

    # Deciding Dataset
    if dataset_name == None:
        if params.dataset == 'cifar10':
            trainset = torchvision.datasets.CIFAR10(root='./data/data-cifar10', train=True,
                                                    download=True, transform=train_transformer)
            devset = torchvision.datasets.CIFAR10(root='./data/data-cifar10', train=False,
                                                  download=True, transform=dev_transformer)
        elif params.dataset == 'cifar100':
            trainset = torchvision.datasets.CIFAR100(root='./data/data-cifar100', train=True,
                                                     download=True, transform=train_transformer)
            devset = torchvision.datasets.CIFAR100(root='./data/data-cifar100', train=False,
                                                   download=True, transform=dev_transformer)
        elif params.dataset == 'mnist':
            trainset = torchvision.datasets.MNIST(root='./data/data-MNIST', train=True,
                                                  download=True, transform=train_transformer)
            devset = torchvision.datasets.MNIST(root='./data/data-MNIST', train=False,
                                                download=True, transform=dev_transformer)
        elif params.dataset == 'tiny_imagenet':
            trainset, devset = get_final_train_and_test_set()


    else:
        print("Dataset Name :  ----------     " + str(dataset_name) + "     ----------")
        if dataset_name == 'cifar10':
            trainset = torchvision.datasets.CIFAR10(root='./data/data-cifar10', train=True,
                                                    download=True, transform=train_transformer)
            devset = torchvision.datasets.CIFAR10(root='./data/data-cifar10', train=False,
                                                  download=True, transform=dev_transformer)
        elif dataset_name == 'cifar100':
            trainset = torchvision.datasets.CIFAR100(root='./data/data-cifar100', train=True,
                                                     download=True, transform=train_transformer)
            devset = torchvision.datasets.CIFAR100(root='./data/data-cifar100', train=False,
                                                   download=True, transform=dev_transformer)
        elif dataset_name == 'mnist':
            trainset = torchvision.datasets.MNIST(root='./data/data-MNIST', train=True,
                                                  download=True, transform=train_transformer)
            devset = torchvision.datasets.MNIST(root='./data/data-MNIST', train=False,
                                                download=True, transform=dev_transformer)
        elif dataset_name == 'tiny_imagenet':
            trainset, devset = get_final_train_and_test_set()
            # data_dir = './data/tiny-imagenet-200/'
            # data_transforms = {
            #     'train': transforms.Compose([
            #         transforms.RandomRotation(20),
            #         transforms.RandomHorizontalFlip(0.5),
            #         transforms.ToTensor(),
            #         transforms.Normalize([0.4802, 0.4481, 0.3975], [0.2302, 0.2265, 0.2262]),
            #     ]),
            #     'val': transforms.Compose([
            #         transforms.ToTensor(),
            #         transforms.Normalize([0.4802, 0.4481, 0.3975], [0.2302, 0.2265, 0.2262]),
            #     ])
            # }
            # train_dir = data_dir + 'train/'
            # test_dir = data_dir + 'val/'
            # trainset = torchvision.datasets.ImageFolder(train_dir, data_transforms['train'])
            # devset = torchvision.datasets.ImageFolder(test_dir, data_transforms['val'])
#     print(trainset[2][0].shape)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=params.batch_size, shuffle=True, num_workers=params.num_workers)
    devloader = torch.utils.data.DataLoader(devset, batch_size=params.batch_size, shuffle=False, num_workers=params.num_workers)
    
    if types == 'train':
        dl = trainloader
    else:
        dl = devloader

    return dl


def fetch_subset_dataloader(types, params):
    """
    Use only a subset of dataset for KD training, depending on params.subset_percent
    """

    # using random crops and horizontal flip for train set
    if params.augmentation == "yes":
        train_transformer = transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),  # randomly flip image horizontally
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))])

    # data augmentation can be turned off
    else:
        train_transformer = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))])

    # transformer for dev set
    dev_transformer = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))])

    if params.dataset == 'cifar10':
        trainset = torchvision.datasets.CIFAR10(root='./data-cifar10', train=True,
                                                download=True, transform=train_transformer)
        devset = torchvision.datasets.CIFAR10(root='./data-cifar10', train=False,
                                              download=True, transform=dev_transformer)
    elif params.dataset == 'cifar100':
        trainset = torchvision.datasets.CIFAR10(root='./data-cifar10', train=True,
                                                download=True, transform=train_transformer)
        devset = torchvision.datasets.CIFAR10(root='./data-cifar10', train=False,
                                              download=True, transform=dev_transformer)
    elif params.dataset == 'tiny_imagenet':
        trainset, devset = get_final_train_and_test_set()
        # data_dir = './data/tiny-imagenet-200/'
        # data_transforms = {
        #     'train': transforms.Compose([
        #         transforms.RandomRotation(20),
        #         transforms.RandomHorizontalFlip(0.5),
        #         transforms.ToTensor(),
        #         transforms.Normalize([0.4802, 0.4481, 0.3975], [0.2302, 0.2265, 0.2262]),
        #     ]),
        #     'val': transforms.Compose([
        #         transforms.ToTensor(),
        #         transforms.Normalize([0.4802, 0.4481, 0.3975], [0.2302, 0.2265, 0.2262]),
        #     ])
        # }
        # train_dir = data_dir + 'train/'
        # test_dir = data_dir + 'val/'
        # trainset = torchvision.datasets.ImageFolder(train_dir, data_transforms['train'])
        # devset = torchvision.datasets.ImageFolder(test_dir, data_transforms['val'])

    trainset_size = len(trainset)
    indices = list(range(trainset_size))
    split = int(np.floor(params.subset_percent * trainset_size))
    np.random.seed(230)
    np.random.shuffle(indices)

    train_sampler = SubsetRandomSampler(indices[:split])

    trainloader = torch.utils.data.DataLoader(trainset, batch_size=params.batch_size,
                                              sampler=train_sampler, num_workers=params.num_workers)

    devloader = torch.utils.data.DataLoader(devset, batch_size=params.batch_size,
                                            shuffle=False, num_workers=params.num_workers)
   # print(trainset[0])
    if types == 'train':
        dl = trainloader
    else:
        dl = devloader

    return dl


In [13]:
##################################################################
#   Evaluate
##################################################################
from sklearn.metrics import f1_score, precision_score, recall_score, cohen_kappa_score, classification_report
"""Evaluates the model"""

import argparse
import logging
import torch
from torch.autograd import Variable


parser = argparse.ArgumentParser()
parser.add_argument('--model_dir', default='experiments/base_model',
                    help="Directory of params.json")
parser.add_argument('--restore_file', default='best', help="name of the file in --model_dir \
                     containing weights to load")

import time
def evaluate(model, loss_fn, dataloader, params, args):
    print('*'*10,'Evaluation', '*'*10)
    """Evaluate the model on `num_steps` batches.

    Args:
        model: (torch.nn.Module) the neural network
        loss_fn: a function that takes batch_output and batch_labels and computes the loss for the batch
        dataloader: (DataLoader) a torch.utils.data.DataLoader object that fetches data
        metrics: (dict) a dictionary of functions that compute a metric using the output and labels of each batch
        params: (Params) hyperparameters
        num_steps: (int) number of batches to train on, each of size params.batch_size
    """
    tracker = CarbonTracker(epochs=100)
    # set model to evaluation mode
    tracker.epoch_start()
    start = time.time()
    model.eval()
    losses = AverageMeter()
    total = 0
    correct = 0
    
    # compute metrics over the dataset
    for data_batch, labels_batch in dataloader:
        data_batch, labels_batch = data_batch.to(device), labels_batch.to(device)

        data_batch, labels_batch = Variable(data_batch), Variable(labels_batch)
        # compute model output
        output_batch = model(data_batch)
        if args.regularization:
            loss = loss_fn(output_batch, labels_batch, params)
        else:
            loss = loss_fn(output_batch, labels_batch)
        
        losses.update(loss.data, data_batch.size(0))
        _, predicted = output_batch.max(1)
        total += labels_batch.size(0)
        correct += predicted.eq(labels_batch).sum().item()
        data_batch.cpu().data.numpy().argmax()
        labels_batch.cpu().data.numpy().argmax()
        data_sc.extends(data_batch)
        data_rc.extends(labels_batch)
    
    loss_avg = losses.avg
    acc = 100.*correct/total
    logging.info(
        "- Eval metrics, acc:{acc:.4f}, loss: {loss_avg:.4f}".format(acc=acc, loss_avg=loss_avg))
    my_metric = {'accuracy': acc, 'loss': loss_avg}
    end = time.time()
    print("Evaluation Time: ", end-start)
    tracker.epoch_end()
    return my_metric


"""
This function duplicates "evaluate()" but ignores "loss_fn" simply for speedup purpose.
Validation loss during KD mode would display '0' all the time.
One can bring that info back by using the fetched teacher outputs during evaluation (refer to train.py)
"""


def evaluate_kd(model, dataloader, params):
    """Evaluate the model on `num_steps` batches.
    
    Args:
        model: (torch.nn.Module) the neural network
        loss_fn: a function that takes batch_output and batch_labels and computes the loss for the batch
        dataloader: (DataLoader) a torch.utils.data.DataLoader object that fetches data
        metrics: (dict) a dictionary of functions that compute a metric using the output and labels of each batch
        params: (Params) hyperparameters
        num_steps: (int) number of batches to train on, each of size params.batch_size
    """

    # set model to evaluation mode
    model.eval()
    total = 0
    correct = 0
    data_sc = []
    data_rc = []

    # compute metrics over the dataset
    for i, (data_batch, labels_batch) in enumerate(dataloader):
        # move to GPU if available
        data_batch, labels_batch = data_batch.to(device), labels_batch.to(device)
        # fetch the next evaluation batch
        data_batch, labels_batch = Variable(data_batch), Variable(labels_batch)

        # compute model output
        output_batch = model(data_batch)
        
        # loss = loss_fn_kd(output_batch, labels_batch, output_teacher_batch, params)
        loss = 0.0  # force validation loss to zero to reduce computation time
        _, predicted = output_batch.max(1)
        total += labels_batch.size(0)
        correct += predicted.eq(labels_batch).sum().item()
        predicted = predicted.cpu().data.numpy()
        labels_batch = labels_batch.cpu().data.numpy()
        data_sc.extend(predicted)
        data_rc.extend(labels_batch)
#     print(data_rc)
    fscore = f1_score(data_rc, data_sc, average = 'weighted')
    precision = precision_score(data_rc, data_sc, average = 'weighted')
    recall = recall_score(data_rc, data_sc, average = 'weighted')
    kappa = cohen_kappa_score(data_rc, data_sc)
    C = classification_report(data_rc, data_sc)
    
    print("Precision: ", precision)
    print("Recall: ", recall)
    print("Report: ", C)
    print("F1: ", fscore)
    print("Kappa: ", kappa)
    print("Prevalence: ", 1000/10000)
    acc = 100. * correct / total
    logging.info("- Eval metrics, acc:{acc:.4f}, loss: {loss:.4f}".format(acc=acc, loss=loss))
    my_metric = {'accuracy': acc, 'loss': loss}
    #my_metric['accuracy'] = acc
    return my_metric


In [14]:
##################################################################
#   My loss functions
##################################################################
import torch
import torch.nn as nn
import torch.nn.functional as F
import random

def loss_kd(outputs, labels, teacher_outputs, params):
    """
    loss function for Knowledge Distillation (KD)
    """
    alpha = 0.95
    T = 20
    loss_CE = F.cross_entropy(outputs, labels)
    D_KL = nn.KLDivLoss()(F.log_softmax(outputs/T, dim=1),
                          F.softmax(teacher_outputs/T, dim=1)) * (T * T)
    KD_loss = (1. - alpha)*loss_CE + alpha*D_KL

    return KD_loss


# def dynamic_rectification(outputs, labels):
#     _, predicted = outputs.max(1)
#     correct = predicted.eq(labels)

#     for i in range(correct.shape[0]):
#         if correct[i].item() == False:
#             c_index = labels[i].item()
#             p_index = predicted[i].item()
#             tmp1, tmp2 = outputs[i, p_index].item(), outputs[i, c_index].item()
#             outputs[i, c_index] = tmp1
#             outputs[i, p_index] = tmp2
    
#     return outputs

def loss_swap(outputs, labels):
    _, predicted = outputs.max(1)
    tmp = outputs[torch.arange(len(labels)), predicted]
    outputs[torch.arange(len(labels)), predicted] = outputs[torch.arange(
        len(labels)), labels]
    outputs[torch.arange(len(labels)), labels] = tmp
    return outputs


def loss_kd_self(outputs, labels, teacher_outputs, params, swap_option='None'):
    """
    loss function for self training: Tf-KD_{self}
    """
    alpha = params.alpha
    T = params.temperature
    """
        Swapping the outputs here
        student_probability_swap || teacher_probability_swap 
    """
    if swap_option == 'student_probability_swap':
        outputs = loss_swap(outputs, labels)
    elif swap_option == 'teacher_probability_swap':
        teacher_outputs = loss_swap(teacher_outputs, labels)

    loss_CE = F.cross_entropy(outputs, labels)
    D_KL = nn.KLDivLoss()(F.log_softmax(outputs/T, dim=1), F.softmax(teacher_outputs/T, dim=1)) * \
        (T * T) * params.multiplier  # multiple is 1.0 in most of cases, some cases are 10 or 50
    KD_loss = (1. - alpha)*loss_CE + alpha*D_KL

    return KD_loss


def loss_kd_regularization(outputs, labels, params):
    """
    loss function for mannually-designed regularization: Tf-KD_{reg}
    """
    alpha = params.reg_alpha
    T = params.reg_temperature
    correct_prob = 0.99    # the probability for correct class in u(k)
    loss_CE = F.cross_entropy(outputs, labels)
    K = outputs.size(1)

    teacher_soft = torch.ones_like(outputs).to(device)
    teacher_soft = teacher_soft*(1-correct_prob)/(K-1)  # p^d(k)
    for i in range(outputs.shape[0]):
        teacher_soft[i, labels[i]] = correct_prob
    loss_soft_regu = nn.KLDivLoss()(F.log_softmax(outputs, dim=1),
                                    F.softmax(teacher_soft/T, dim=1))*params.multiplier

    KD_loss = (1. - alpha)*loss_CE + alpha*loss_soft_regu

    return KD_loss


def loss_kd_self_plus_regularization(outputs, labels, teacher_outputs, params, swap_option='None'):

    new_alpha = 0.45
   
    #loss function for self training: Tf-KD_{self}
    alpha_self = params.alpha
    T = params.temperature
    """
        Swapping the outputs here
        student_probability_swap || teacher_probability_swap 
    """
    if swap_option == 'student_probability_swap':
        outputs = loss_swap(outputs, labels)
    elif swap_option == 'teacher_probability_swap':
        teacher_outputs = loss_swap(teacher_outputs, labels)

    loss_CE = F.cross_entropy(outputs, labels)
    D_KL_Self = nn.KLDivLoss()(F.log_softmax(outputs/T, dim=1), F.softmax(teacher_outputs/T, dim=1)) * \
        (T * T) * params.multiplier  # multiple is 1.0 in most of cases, some cases are 10 or 50
    


    #loss function for mannually-designed regularization: Tf-KD_{reg}
    alpha_reg = params.reg_alpha
    T = params.reg_temperature
    correct_prob = 0.99    # the probability for correct class in u(k)
    loss_CE = F.cross_entropy(outputs, labels)
    K = outputs.size(1)

    teacher_soft = torch.ones_like(outputs).to(device)
    teacher_soft = teacher_soft*(1-correct_prob)/(K-1)  # p^d(k)
    for i in range(outputs.shape[0]):
        teacher_soft[i, labels[i]] = correct_prob
    loss_soft_regu = nn.KLDivLoss()(F.log_softmax(outputs, dim=1),F.softmax(teacher_soft/T, dim=1))*params.multiplier



    # final loss 
    KD_loss = (1. - alpha_reg)*loss_CE + new_alpha*loss_soft_regu + new_alpha*D_KL_Self

    return KD_loss





def loss_label_smoothing(outputs, labels):
    """
    loss function for label smoothing regularization
    """
    alpha = 0.1
    N = outputs.size(0)  # batch_size
    C = outputs.size(1)  # number of classes
    smoothed_labels = torch.full(
        size=(N, C), fill_value=alpha / (C - 1)).to(device)
    smoothed_labels.scatter_(
        dim=1, index=torch.unsqueeze(labels, dim=1), value=1-alpha)

    log_prob = torch.nn.functional.log_softmax(outputs, dim=1)
    loss = -torch.sum(log_prob * smoothed_labels) / N

    return loss


In [15]:
##################################################################
#   Train functions
##################################################################


import os
import time
import math
from tqdm import tqdm
import logging
from torch.autograd import Variable
from torch.utils.tensorboard import SummaryWriter
from torch.optim.lr_scheduler import StepLR, MultiStepLR


# KD train and evaluate
def train_and_evaluate_kd(model, teacher_model, train_dataloader, val_dataloader, optimizer,
                       loss_fn_kd, warmup_scheduler, params, args, restore_file=None):
    """
    KD Train the model and evaluate every epoch.
    """
    # reload weights from restore_file if specified
    if restore_file is not None:
        restore_path = os.path.join(args.model_dir, args.restore_file + '.pth.tar')
        logging.info("Restoring parameters from {}".format(restore_path))
        load_checkpoint(restore_path, model, optimizer)

    # tensorboard setting
    log_dir = args.model_dir + '/tensorboard/'
    writer = SummaryWriter(log_dir=log_dir)
    best_val_acc = 0.0
    teacher_model.eval()
    teacher_acc = evaluate_kd(teacher_model, val_dataloader, params)
    print(">>>>>>>>>The teacher accuracy: {}>>>>>>>>>".format(teacher_acc['accuracy']))

    scheduler = MultiStepLR(optimizer, milestones=[30, 60, 80], gamma=0.2)
    for epoch in range(params.num_epochs):
        if epoch > 0:   # 0 is the warm up epoch
            scheduler.step()
        logging.info("Epoch {}/{}, lr:{}".format(epoch + 1, params.num_epochs, optimizer.param_groups[0]['lr']))

        # KD Train
        train_acc, train_loss = train_kd(model, teacher_model, optimizer, loss_fn_kd, train_dataloader, warmup_scheduler, params, args, epoch)
        # Evaluate
        val_metrics = evaluate_kd(model, val_dataloader, params)

        val_acc = val_metrics['accuracy']
        is_best = val_acc>=best_val_acc

        # Save weights
        save_checkpoint({'epoch': epoch + 1,
                               'state_dict': model.state_dict(),
                               'optim_dict' : optimizer.state_dict()},
                               is_best=is_best,
                               checkpoint=args.model_dir)

        # If best_eval, best_save_path
        if is_best:
            logging.info("*********** Hurray ! Found new best accuracy *****************")
            best_val_acc = val_acc

            # Save best val metrics in a json file in the model directory
            file_name = "eval_best_result.json"
            best_json_path = os.path.join(args.model_dir, file_name)
            save_dict_to_json(val_metrics, best_json_path)

        # Save latest val metrics in a json file in the model directory
        last_json_path = os.path.join(args.model_dir, "eval_last_result.json")
        save_dict_to_json(val_metrics, last_json_path)

        # Tensorboard
        writer.add_scalar('Train_accuracy', train_acc, epoch)
        writer.add_scalar('Train_loss', train_loss, epoch)
        writer.add_scalar('Test_accuracy', val_metrics['accuracy'], epoch)
        writer.add_scalar('Test_loss', val_metrics['loss'], epoch)
        # export scalar data to JSON for external processing
    writer.close()


# Defining train_kd functions
def train_kd(model, teacher_model, optimizer, loss_fn_kd, dataloader, warmup_scheduler, params, args, epoch, flag=None):
    """
    KD Train the model on `num_steps` batches
    """
    # set model to training mode
    model.train()
    teacher_model.eval()
    loss_avg = RunningAverage()
    losses = AverageMeter()
    total = 0
    correct = 0
    # Use tqdm for progress bar
    with tqdm(total=len(dataloader)) as t:
        for i, (train_batch, labels_batch) in enumerate(dataloader):
            if epoch<=0:
                warmup_scheduler.step()

            train_batch, labels_batch = train_batch.to(device), labels_batch.to(device)
            # convert to torch Variables
            train_batch, labels_batch = Variable(train_batch), Variable(labels_batch)

            # compute model output, fetch teacher output, and compute KD loss
            output_batch = model(train_batch)

            # get one batch output from teacher model
            output_teacher_batch = teacher_model(train_batch).to(device)
            output_teacher_batch = Variable(output_teacher_batch, requires_grad=False)

            loss = loss_fn_kd(output_batch, labels_batch, output_teacher_batch, params)

            # clear previous gradients, compute gradients of all variables wrt loss
            optimizer.zero_grad()
            loss.backward()

            # performs updates using calculated gradients
            optimizer.step()

            _, predicted = output_batch.max(1)
            total += labels_batch.size(0)
            correct += predicted.eq(labels_batch).sum().item()
            # update the average loss
            loss_avg.update(loss.data)
            losses.update(loss.item(), train_batch.size(0))

            t.set_postfix(loss='{:05.3f}'.format(loss_avg()), lr='{:05.6f}'.format(optimizer.param_groups[0]['lr']))
            t.update()

    acc = 100.*correct/total
    logging.info("- Train accuracy: {acc:.4f}, training loss: {loss:.4f}".format(acc = acc, loss = losses.avg))
    return acc, losses.avg


# normal training
def train_and_evaluate(model, train_dataloader, val_dataloader, optimizer,
                       loss_fn, params, model_dir, warmup_scheduler, args, restore_file=None):
    """
    Train the model and evaluate every epoch.
    """
    # reload weights from restore_file if specified
    if restore_file is not None:
        restore_path = os.path.join(args.model_dir, args.restore_file + '.pth.tar')
        logging.info("Restoring parameters from {}".format(restore_path))
        load_checkpoint(restore_path, model, optimizer)

    # dir setting, tensorboard events will save in the dirctory
    log_dir = args.model_dir + '/base_train/'
    if args.regularization:
        log_dir = args.model_dir + '/Tf-KD_regularization/'
        model_dir = log_dir
    elif args.label_smoothing:
        log_dir = args.model_dir + '/label_smoothing/'
        model_dir = log_dir
    writer = SummaryWriter(log_dir=log_dir)

    best_val_acc = 0.0

    # learning rate schedulers
    scheduler = MultiStepLR(optimizer, milestones=[30, 60, 80], gamma=0.2)
    
    for epoch in range(params.num_epochs):

        if epoch > 0:   # 1 is the warm up epoch
            scheduler.step(epoch)

        # Run one epoch
        logging.info("Epoch {}/{}, lr:{}".format(epoch + 1, params.num_epochs, optimizer.param_groups[0]['lr']))

        # compute number of batches in one epoch (one full pass over the training set)
        train_acc, train_loss = train(model, optimizer, loss_fn, train_dataloader, params, epoch, warmup_scheduler, args)

        # Evaluate for one epoch on validation set
        val_metrics = evaluate(model, loss_fn, val_dataloader, params, args)

        val_acc = val_metrics['accuracy']
        is_best = val_acc>=best_val_acc

        # Save weights
        save_checkpoint({'epoch': epoch + 1,
                               'state_dict': model.state_dict(),
                               'optim_dict' : optimizer.state_dict()},
                                is_best=is_best,
                                checkpoint=model_dir)
        # If best_eval, best_save_path
        if is_best:
            logging.info("- Found new best accuracy")
            best_val_acc = val_acc

            # Save best val metrics in a json file in the model directory
            best_json_path = os.path.join(model_dir, "eval_best_results.json")
            save_dict_to_json(val_metrics, best_json_path)

        # Save latest val metrics in a json file in the model directory
        last_json_path = os.path.join(model_dir, "eval_last_results.json")
        save_dict_to_json(val_metrics, last_json_path)

        # Tensorboard
        writer.add_scalar('Train_accuracy', train_acc, epoch)
        writer.add_scalar('Train_loss', train_loss, epoch)
        writer.add_scalar('Test_accuracy', val_metrics['accuracy'], epoch)
        writer.add_scalar('Test_loss', val_metrics['loss'], epoch)

    writer.close()


# normal training function
def train(model, optimizer, loss_fn, dataloader, params, epoch, warmup_scheduler, args):
    """
    Noraml training, without KD
    """

    # set model to training mode
    model.train()
    loss_avg = RunningAverage()
    losses = AverageMeter()
    total = 0
    correct = 0
    # Use tqdm for progress bar
    with tqdm(total=len(dataloader)) as t:
        for i, data in enumerate(dataloader):
            #print(data)
#             print(i)
            train_batch, labels_batch = data
            train_batch, labels_batch = train_batch.cuda(), labels_batch.cuda()
            if epoch<=0:
                warmup_scheduler.step()
            train_batch, labels_batch = Variable(train_batch), Variable(labels_batch)

            optimizer.zero_grad()
            output_batch = model(train_batch)
            
            if args.regularization:
                loss = loss_fn(output_batch, labels_batch, params)
            else:
                loss = loss_fn(output_batch, labels_batch)
            loss.backward()
            optimizer.step()

            _, predicted = output_batch.max(1)
            total += labels_batch.size(0)
            correct += predicted.eq(labels_batch).sum().item()

            # update the average loss
            loss_avg.update(loss.data)
            losses.update(loss.data, train_batch.size(0))

            t.set_postfix(loss='{:05.3f}'.format(loss_avg()), lr='{:05.6f}'.format(optimizer.param_groups[0]['lr']))
            t.update()

    acc = 100. * correct / total
    logging.info("- Train accuracy: {acc: .4f}, training loss: {loss: .4f}".format(acc=acc, loss=losses.avg))
    return acc, losses.avg


In [16]:
def read_params_json(json_path):
    f = open(json_path)
    json_object = json.load(f)
    print("################## Json Params ##################")
    json_formatted_str = json.dumps(json_object, indent=2)
    print(json_formatted_str)

In [17]:
import torchvision
import torch.nn as nn
from torchvision import datasets, models, transforms
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet50":
        """ Resnet50
        """
        model_ft = models.resnet50(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224


    elif model_name == "densenet121":
        """ Densenet121
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224


    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft



In [18]:
 class ArmentPassingClass:
  def __init__(self, model_dir, restore_file, num_class, warm, regularization, label_smoothing, double_training, 
               self_training, swap_option, dataset_name, random_seeder, pt_teacher):
    self.model_dir = model_dir
    self.restore_file = restore_file
    self.num_class = num_class
    self.warm = warm
    self.regularization = regularization
    self.label_smoothing = label_smoothing
    self.double_training = double_training
    self.self_training = self_training
    self.swap_option = swap_option
    self.dataset_name = dataset_name
    self.random_seeder = random_seeder
    self.pt_teacher = pt_teacher

dataset_num_class_dict = {'cifar10': 10, 'cifar100': 100, 'tiny_imagenet':200}


############# Dataset Assigning #############
dataset_name = 'cifar10' 
num_class = dataset_num_class_dict[dataset_name]
restore_file = None


############# KD Train #############
# model_dir =  'experiments/kd_experiments/shufflenet_distill/resnet18_teacher/' + dataset_name

# model_dir =  'experiments/kd_experiments/resnet50_distill/resnet50_self_teacher/' + dataset_name
# model_dir =  'experiments/kd_experiments/densenet121_distill/densenet_self_teacher/' + dataset_name
# model_dir =  'experiments/kd_experiments/googlenet_distill/googlenet_self_teacher/' + dataset_name
model_dir =  'experiments/kd_experiments/mobilenet_distill/resnet18_teacher/' + dataset_name
self_training = False
swap_option = 'teacher_probability_swap'



############# Base Train #############  
# model_dir =  'experiments/base_experiments/base_shufflenetv2/' + dataset_name
# model_dir =  'experiments/base_experiments/base_resnet18/' + dataset_name
# # model_dir =  'experiments/base_experiments/base_mobilenetv2/' + dataset_name
# self_training = False
# swap_option = None




args = ArmentPassingClass(  model_dir = model_dir, 
                            restore_file = restore_file, 
                            num_class = num_class, 
                            warm = 1, 
                            regularization = False, 
                            label_smoothing = False, 
                            double_training = False, 
                            self_training = self_training, 
                            swap_option = swap_option, 
                            dataset_name = dataset_name, 
                            random_seeder = 2320, 
                            pt_teacher = False)

In [24]:
class QuantizedResNet18(nn.Module):
    def __init__(self, model_fp32):

        super(QuantizedResNet18, self).__init__()
        # QuantStub converts tensors from floating point to quantized.
        # This will only be used for inputs.
        self.quant = torch.quantization.QuantStub()
        # DeQuantStub converts tensors from quantized to floating point.
        # This will only be used for outputs.
        self.dequant = torch.quantization.DeQuantStub()
        # FP32 model
        self.model_fp32 = model_fp32

    def forward(self, x):
        # manually specify where tensors will be converted from floating
        # point to quantized in the quantized model
        x = self.quant(x)
        x = self.model_fp32(x)
        # manually specify where tensors will be converted from quantized
        # to floating point in the quantized model
        x = self.dequant(x)
        return x
    
def model_equivalence(model_1,
                      model_2,
                      device,
                      rtol=1e-05,
                      atol=1e-08,
                      num_tests=100,
                      input_size=(1, 3, 32, 32)):

    model_1.to(device)
    model_2.to(device)

    for _ in range(num_tests):
        x = torch.rand(size=input_size).to(device)
        y1 = model_1(x).detach().cpu().numpy()
        y2 = model_2(x).detach().cpu().numpy()
        if np.allclose(a=y1, b=y2, rtol=rtol, atol=atol,
                       equal_nan=False) == False:
            print("Model equivalence test sample failed: ")
            print(y1)
            print(y2)
            return False

    return True
def calibrate_model(model, loader, device=torch.device("cpu:0")):

    model.to(device)
    model.eval()

    for inputs, labels in loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        _ = model(inputs)


def measure_inference_latency(model,
                              device,
                              input_size=(1, 3, 32, 32),
                              num_samples=100,
                              num_warmups=10):

    model.to(device)
    model.eval()

    x = torch.rand(size=input_size).to(device)

    with torch.no_grad():
        for _ in range(num_warmups):
            _ = model(x)
    torch.cuda.synchronize()

    with torch.no_grad():
        start_time = time.time()
        for _ in range(num_samples):
            _ = model(x)
            torch.cuda.synchronize()
        end_time = time.time()
    elapsed_time = end_time - start_time
    elapsed_time_ave = elapsed_time / num_samples

    return elapsed_time_ave

In [19]:
##################################################################
#   Main
##################################################################


"""
Teacher free KD, main.py
"""
import argparse
import logging
import os
import sys
import random
import warnings



import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchsummary import summary

# import data_loader as data_loader
import model.alexnet as alexnet
import model.densenet as densenet
import model.googlenet as googlenet
import model.mobilenetv2 as mobilenet
import model.net as net
import model.resnet as resnet
import model.resnext as resnext
import model.shufflenetv2 as shufflenet
# import utils
# from my_loss_function import (loss_kd, loss_kd_regularization, loss_kd_self,
#                               loss_label_smoothing)
# from train_kd import train_and_evaluate, train_and_evaluate_kd
from pprint import pprint


def main():
    # Load the parameters from json file
    # args = parser.parse_args(argv[1:])
    # args = parser.parse_args()
    torch.cuda.empty_cache()
    json_path = os.path.join(args.model_dir.rsplit('/',1)[0] , args.dataset_name + '_params.json')
    assert os.path.isfile(json_path), "No json configuration file found at {}".format(json_path)
    params = Params(json_path)
    params.num_epochs = 100
#     params.batch_size = bs
#     params.dataset = 'tiny_imagenet'
    read_params_json(json_path)
    pprint(vars(params))
    # Set the random seed for reproducible experiments
    random_seeder = int(args.random_seeder)
    print('Random Seeder :  {}'.format(random_seeder))
    print('Dataset Name : ', args.dataset_name, '  Number of classes: ', args.num_class)
    random.seed(random_seeder)
    torch.manual_seed(random_seeder)
    np.random.seed(random_seeder)
    torch.cuda.manual_seed(random_seeder)
    warnings.filterwarnings("ignore")

    # Set the logger
    
    if not os.path.exists(args.model_dir):
        os.makedirs(args.model_dir)
    logger_path = os.path.join(args.model_dir, 'train.log')  
    set_logger(logger_path)
    
    # Create the input data pipeline
    logging.info("Loading the datasets...")

    # fetch dataloaders, considering full-set vs. sub-set scenarios

    if params.subset_percent < 1.0:
        train_dl = fetch_subset_dataloader('train', params)
    else:
        train_dl = fetch_dataloader('train', params, args.dataset_name)
    
    dev_dl = fetch_dataloader('dev', params, args.dataset_name)

    logging.info("- done.")

    """
    Load student and teacher model
    """
    if "distill" in params.model_version:
      
        # Specify the student models
        if params.model_version == "cnn_distill":  # 5-layers Plain CNN
            print("Student model: {}".format(params.model_version))
            model = net.Net(params).to(device)

        elif params.model_version == "shufflenet_v2_distill":
            print("Student model: {}".format(params.model_version))
            model = shufflenet.shufflenetv2(class_num=args.num_class).to(device)

        elif params.model_version == "mobilenet_v2_distill":
            print("Student model: {}".format(params.model_version))
            model = mobilenet.mobilenetv2(class_num=args.num_class).to(device)

        elif params.model_version == 'resnet18_distill':
            print("Student model: {}".format(params.model_version))
            model = resnet.ResNet18(num_classes=args.num_class).to(device)

        elif params.model_version == 'resnet50_distill':
            print("Student model: {}".format(params.model_version))
            model_name  = 'resnet50'
            num_classes = args.num_class
            feature_extract=False
            model = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)
            model = model.to(device)

        elif params.model_version == "alexnet_distill":
            print("Student model: {}".format(params.model_version))
            model = alexnet.alexnet(num_classes=args.num_class).to(device)

        elif params.model_version == "vgg19_distill":
            print("Student model: {}".format(params.model_version))
            model = models.vgg19_bn(num_classes=args.num_class).to(device)

        elif params.model_version == "googlenet_distill":
            print("Student model: {}".format(params.model_version))
            model = googlenet.GoogleNet(num_class=args.num_class).to(device)

        elif params.model_version == "resnext29_distill":
            print("Student model: {}".format(params.model_version))
            model = resnext.CifarResNeXt(
                cardinality=8, depth=29, num_classes=args.num_class).to(device)

        elif params.model_version == "densenet121_distill":
            print("Student model: {}".format(params.model_version))
            model_name  = 'densenet121'
            num_classes = args.num_class
            feature_extract=False
            model = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)
            model = model.to(device)
            #model = densenet.densenet121(num_class=args.num_class).to(device)

        # optimizer
        if params.model_version == "cnn_distill":
            optimizer = optim.Adam(model.parameters(), lr=params.learning_rate * (params.batch_size / 128))
        else:
            optimizer = optim.SGD(model.parameters(), lr=params.learning_rate * (params.batch_size / 128), momentum=0.9,
                                  weight_decay=5e-4)

        iter_per_epoch = len(train_dl)
        warmup_scheduler = WarmUpLR(optimizer,iter_per_epoch * args.warm)  # warmup the learning rate in the first epoch

        # specify loss function
        if args.self_training:
            print('>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>self training>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
            if args.swap_option == 'student_probability_swap':
                print('------------Swapping Student Output------------')
            elif args.swap_option == 'teacher_probability_swap':
                print('------------Swapping Teacher Output------------')
            loss_fn_kd = loss_kd_self
        else:
            loss_fn_kd = loss_kd

        """ 
            Specify the pre-trained teacher models for knowledge distillation
            Checkpoints can be obtained by regular training or downloading our pretrained models
            For model which is pretrained in multi-GPU, use "nn.DaraParallel" to correctly load the model weights.
        """
        
        model_name_plus_best_pth_tar = args.dataset_name + '/best.pth.tar'
        if params.teacher == "resnet18":
            print("Teacher model: {}".format(params.teacher))
            teacher_model = resnet.ResNet18(num_classes=args.num_class)
            teacher_checkpoint = 'experiments/pretrained_teacher_models/base_resnet18/' + model_name_plus_best_pth_tar
            if args.pt_teacher:  # poorly-trained teacher for Defective KD experiments
                teacher_checkpoint = 'experiments/pretrained_teacher_models/base_resnet18/0.pth.tar'
            teacher_model = teacher_model.to(device)

        elif params.teacher == "alexnet":
            print("Teacher model: {}".format(params.teacher))
            teacher_model = alexnet.alexnet(num_classes=args.num_class)
            teacher_checkpoint = 'experiments/pretrained_teacher_models/base_alexnet/' + model_name_plus_best_pth_tar
            teacher_model = teacher_model.to(device)

        elif params.teacher == "googlenet":
            print("Teacher model: {}".format(params.teacher))
            teacher_model = googlenet.GoogleNet(num_class=args.num_class)
            teacher_checkpoint = 'experiments/pretrained_teacher_models/base_googlenet/' + model_name_plus_best_pth_tar
            teacher_model = teacher_model.to(device)

        elif params.teacher == "vgg19":
            print("Teacher model: {}".format(params.teacher))
            teacher_model = models.vgg19_bn(num_classes=args.num_class)
            teacher_checkpoint = 'experiments/pretrained_teacher_models/base_vgg19/' + model_name_plus_best_pth_tar
            teacher_model = teacher_model.to(device)

        elif params.teacher == "resnet50":
            print("Teacher model: {}".format(params.teacher))
            #teacher_model = resnet.ResNet50(num_classes=args.num_class).to(device)
            model_name  = params.teacher
            num_classes = args.num_class
            feature_extract=False
            teacher_model = initialize_model(model_name, num_classes, feature_extract, use_pretrained=False)
            teacher_model = teacher_model.to(device)
            teacher_checkpoint = 'experiments/pretrained_teacher_models/base_resnet50/' + model_name_plus_best_pth_tar
            if args.pt_teacher:  # poorly-trained teacher for Defective KD experiments
                teacher_checkpoint = 'experiments/pretrained_teacher_models/base_resnet50/50.pth.tar'

        elif params.teacher == "densenet121":
            print("Teacher model: {}".format(params.teacher))
            #teacher_model = densenet.densenet121(num_class=args.num_class).to(device)
            model_name  = params.teacher
            num_classes = args.num_class
            feature_extract=False
            teacher_model = initialize_model(model_name, num_classes, feature_extract, use_pretrained=False)
            teacher_model = teacher_model.to(device)
            teacher_checkpoint = 'experiments/pretrained_teacher_models/base_densenet121/' + model_name_plus_best_pth_tar
            # teacher_model = nn.DataParallel(teacher_model).to(device)

        elif params.teacher == "resnet101":
            print("Teacher model: {}".format(params.teacher))
            teacher_model = resnet.ResNet101(num_classes=args.num_class).to(device)
            teacher_checkpoint = 'experiments/pretrained_teacher_models/base_resnet101/' + model_name_plus_best_pth_tar
            teacher_model = teacher_model.to(device)

        elif params.teacher == "resnext29":
            print("Teacher model: {}".format(params.teacher))
            teacher_model = resnext.CifarResNeXt(cardinality=8, depth=29, num_classes=args.num_class).to(device)
            teacher_checkpoint = 'experiments/pretrained_teacher_models/base_resnext29/' + model_name_plus_best_pth_tar
            if args.pt_teacher:  # poorly-trained teacher for Defective KD experiments
                teacher_checkpoint = 'experiments/pretrained_teacher_models/base_resnext29/50.pth.tar'
                teacher_model = nn.DataParallel(teacher_model).to(device)

        elif params.teacher == "mobilenet_v2":
            print("Teacher model: {}".format(params.teacher))
            teacher_model = mobilenet.mobilenetv2(class_num=args.num_class).to(device)
            teacher_checkpoint = 'experiments/pretrained_teacher_models/base_mobilenet_v2/' + model_name_plus_best_pth_tar
        
            
            

        elif params.teacher == "shufflenet_v2":
            print("Teacher model: {}".format(params.teacher))
            teacher_model = shufflenet.shufflenetv2(class_num=args.num_class).to(device)
            teacher_checkpoint = 'experiments/pretrained_teacher_models/base_shufflenet_v2/' + model_name_plus_best_pth_tar
            
            
        print('Teacher checkpoint directory : ', teacher_checkpoint)
        load_checkpoint(teacher_checkpoint, teacher_model)

        # Train the model with KD
        logging.info("Starting training for {} epoch(s)".format(params.num_epochs))
        train_and_evaluate_kd(model, teacher_model, train_dl, dev_dl, optimizer, loss_fn_kd,warmup_scheduler, params, args, args.restore_file)

    # non-KD mode: regular training to obtain a baseline model
    else:
        print("Train base model")
        if params.model_version == "cnn":
            model = net.Net(params).to(device)

        elif params.model_version == "mobilenet_v2":
            print("model: {}".format(params.model_version))
            model = mobilenet.mobilenetv2(class_num=args.num_class).to(device)
            print(next(model.parameters()).is_cuda)

        elif params.model_version == "shufflenet_v2":
            print("model: {}".format(params.model_version))
            model = shufflenet.shufflenetv2(class_num=args.num_class).to(device)

        elif params.model_version == "alexnet":
            print("model: {}".format(params.model_version))
            model = alexnet.alexnet(num_classes=args.num_class).to(device)

        elif params.model_version == "vgg19":
            print("model: {}".format(params.model_version))
            model = models.vgg19_bn(num_classes=args.num_class).to(device)

        elif params.model_version == "googlenet":
            print("model: {}".format(params.model_version))
            model = googlenet.GoogleNet(num_class=args.num_class).to(device)

        elif params.model_version == "densenet121":
            print("model: {}".format(params.model_version))
            # model_name  = params.model_version
            # num_classes = args.num_class
            # feature_extract=False
            # model = initialize_model(model_name, num_classes, feature_extract, use_pretrained=False)
            # model = model.to(device)
            # model = densenet.densenet121(num_class=args.num_class).to(device)
            model = densenet121(num_class=args.num_class).to(device)

        elif params.model_version == "resnet18":
            model = resnet.ResNet18(num_classes=args.num_class).to(device)

        elif params.model_version == "resnet50":
            print("model: {}".format(params.model_version))
            # model_name  = 'resnet50'
            # num_classes = args.num_class
            # feature_extract=False
            # model = initialize_model(model_name, num_classes, feature_extract, use_pretrained=False)
            # model = model.to(device)
        
            
            model = resnet.ResNet50(num_classes=args.num_class).to(device)

        elif params.model_version == "resnet101":
            model = resnet.ResNet101(num_classes=args.num_class).to(device)

        elif params.model_version == "resnet152":
            model = resnet.ResNet152(num_classes=args.num_class).to(device)

        elif params.model_version == "resnext29":
            model = resnext.CifarResNeXt(
                cardinality=8, depth=29, num_classes=args.num_class).to(device)
            # model = nn.DataParallel(model).to(device)

        if args.regularization:
            print(">>>>>>>>>>>>>>>>>>>>>>>>Loss of Regularization>>>>>>>>>>>>>>>>>>>>>>>>")
            loss_fn = loss_kd_regularization
        elif args.label_smoothing:
            print(">>>>>>>>>>>>>>>>>>>>>>>>Label Smoothing>>>>>>>>>>>>>>>>>>>>>>>>")
            loss_fn = loss_label_smoothing
        else:
            print(">>>>>>>>>>>>>>>>>>>>>>>>Normal Training>>>>>>>>>>>>>>>>>>>>>>>>")
            loss_fn = nn.CrossEntropyLoss()
            if args.double_training:  # double training, compare to self-KD
                print(">>>>>>>>>>>>>>>>>>>>>>>>Double Training>>>>>>>>>>>>>>>>>>>>>>>>")
                checkpoint = 'experiments/pretrained_teacher_models/base_' + str(params.model_version) + '/best.pth.tar'
                load_checkpoint(checkpoint, model)

        if params.model_version == "cnn":
            optimizer = optim.Adam( model.parameters(), lr=params.learning_rate * (params.batch_size / 128))
        else:
            optimizer = optim.SGD(model.parameters(), lr=params.learning_rate * (params.batch_size / 128), momentum=0.9,weight_decay=5e-4)


        iter_per_epoch = len(train_dl)
        warmup_scheduler = WarmUpLR(optimizer, iter_per_epoch * args.warm)
        # Train the model
        logging.info("Starting training for {} epoch(s)".format(params.num_epochs))
        train_and_evaluate(model, train_dl, dev_dl, optimizer, loss_fn, params, args.model_dir, warmup_scheduler, args, args.restore_file)


In [22]:
if __name__ == "__main__":
    for i in range(0,5):
        main()


Loading the datasets...


################## Json Params ##################
{
  "model_version": "mobilenet_v2_distill",
  "subset_percent": 1.0,
  "augmentation": "yes",
  "teacher": "resnet18",
  "alpha": 0.95,
  "temperature": 20,
  "learning_rate": 0.1,
  "batch_size": 128,
  "num_epochs": 200,
  "dropout_rate": 0.5,
  "save_summary_steps": 100,
  "num_workers": 4,
  "dataset": "cifar100"
}
{'alpha': 0.95,
 'augmentation': 'yes',
 'batch_size': 128,
 'dataset': 'cifar100',
 'dropout_rate': 0.5,
 'learning_rate': 0.1,
 'model_version': 'mobilenet_v2_distill',
 'num_epochs': 100,
 'num_workers': 4,
 'save_summary_steps': 100,
 'subset_percent': 1.0,
 'teacher': 'resnet18',
 'temperature': 20}
Random Seeder :  2320
Dataset Name :  cifar10   Number of classes:  10
Dataset Name :  ----------     cifar10     ----------
Files already downloaded and verified
Files already downloaded and verified
Dataset Name :  ----------     cifar10     ----------
Files already downloaded and verified
Files already downloaded and 

- done.


Student model: mobilenet_v2_distill
Teacher model: resnet18
Teacher checkpoint directory :  experiments/pretrained_teacher_models/base_resnet18/cifar10/best.pth.tar


Starting training for 100 epoch(s)
- Eval metrics, acc:94.9600, loss: 0.0000
Epoch 1/100, lr:0.0


...

Precision:  0.9158831559618394
Recall:  0.9154
Report:                precision    recall  f1-score   support

           0       0.92      0.92      0.92      1000
           1       0.96      0.97      0.96      1000
           2       0.90      0.88      0.89      1000
           3       0.80      0.85      0.83      1000
           4       0.90      0.92      0.91      1000
           5       0.89      0.84      0.86      1000
           6       0.94      0.94      0.94      1000
           7       0.96      0.93      0.94      1000
           8       0.94      0.96      0.95      1000
           9       0.95      0.95      0.95      1000

    accuracy                           0.92     10000
   macro avg       0.92      0.92      0.92     10000
weighted avg       0.92      0.92      0.92     10000

F1:  0.9154596521056712
Kappa:  0.906
Prevalence:  0.1
Checkpoint Directory exists! 


100%|███████████████████████████████████████████████████████| 391/391 [00:30<00:00, 12.88it/s, loss=0.065, lr=0.000800]
- Train accuracy: 94.6960, training loss: 0.0652
- Eval metrics, acc:91.2500, loss: 0.0000
Epoch 100/100, lr:0.0007999999999795398


Precision:  0.9131901268010044
Recall:  0.9125
Report:                precision    recall  f1-score   support

           0       0.91      0.92      0.92      1000
           1       0.96      0.96      0.96      1000
           2       0.92      0.87      0.89      1000
           3       0.80      0.84      0.82      1000
           4       0.90      0.92      0.91      1000
           5       0.87      0.85      0.86      1000
           6       0.93      0.94      0.94      1000
           7       0.96      0.92      0.94      1000
           8       0.93      0.96      0.94      1000
           9       0.95      0.94      0.95      1000

    accuracy                           0.91     10000
   macro avg       0.91      0.91      0.91     10000
weighted avg       0.91      0.91      0.91     10000

F1:  0.9126209996013449
Kappa:  0.9027777777777778
Prevalence:  0.1
Checkpoint Directory exists! 


100%|███████████████████████████████████████████████████████| 391/391 [00:30<00:00, 12.99it/s, loss=0.066, lr=0.000800]
- Train accuracy: 94.5840, training loss: 0.0657
- Eval metrics, acc:91.2700, loss: 0.0000


Precision:  0.9131715867614985
Recall:  0.9127
Report:                precision    recall  f1-score   support

           0       0.92      0.92      0.92      1000
           1       0.96      0.97      0.96      1000
           2       0.89      0.88      0.89      1000
           3       0.79      0.84      0.82      1000
           4       0.92      0.90      0.91      1000
           5       0.89      0.83      0.86      1000
           6       0.92      0.95      0.94      1000
           7       0.95      0.94      0.94      1000
           8       0.95      0.95      0.95      1000
           9       0.94      0.95      0.95      1000

    accuracy                           0.91     10000
   macro avg       0.91      0.91      0.91     10000
weighted avg       0.91      0.91      0.91     10000

F1:  0.9127131243778082
Kappa:  0.903
Prevalence:  0.1
Checkpoint Directory exists! 
